# Make latex tables for Supplement: 

In [6]:
import numpy as np
import pandas as pd
import os
from estimation import *

## Comparing abrupt CO2 forcings

In [7]:
exp = 'abrupt-2xCO2'
best_parameters = pd.read_csv('../Estimates/best_estimated_parameters_allmembers2xCO2.csv', index_col=0)
models2x = list(best_parameters.index)
models2x

['CESM2',
 'CNRM-CM6-1',
 'CanESM5',
 'GISS-E2-1-G',
 'GISS-E2-1-H',
 'GISS-E2-2-G',
 'GISS-E2-2-H',
 'IPSL-CM6A-LR',
 'MIROC6',
 'MRI-ESM2-0',
 'TaiESM1',
 'HadGEM3-GC31-LL']

In [8]:
F2x = pd.DataFrame(best_parameters['F2x'])
F2x.index.name = 'Models'

best_parameters_0p5xCO2 = pd.read_csv('../Estimates/best_estimated_parameters_allmembers0p5xCO2.csv', index_col=0)
F0p5x = pd.DataFrame(- best_parameters_0p5xCO2['F2x']).rename(columns={"F2x": "F0p5x"})
F0p5x.index.name = 'Models'

best_parameters_4xCO2 = pd.read_csv('../Estimates/best_estimated_parameters_allmembers.csv', index_col=0)
F4x = pd.DataFrame(2*best_parameters_4xCO2['F2x']).rename(columns={"F2x": "F4x"})
F4x.index.name = 'Models'

dfs = [F2x, F0p5x, F4x.loc[models2x]]
#dfs = [df.set_index('Model') for df in dfs]
forcing_table = dfs[0].join(dfs[1:])
#forcing_table = pd.merge((F2x, F0p5x, F4x[models2x]))

ratio1 = pd.DataFrame(forcing_table['F4x']/forcing_table['F2x'], columns = ['F4x/F2x'])
ratio2 = pd.DataFrame(forcing_table['F4x']/forcing_table['F0p5x'], columns = ['F4x/F0p5x'])

forcing_table = forcing_table.join([ratio1, ratio2])

# add model mean row in the bottom:
model_means = [forcing_table.iloc[:,i].mean() for i in range(0,len(forcing_table.columns))]
forcing_table = pd.concat([forcing_table,pd.DataFrame([model_means], index = ['Model mean'], columns = forcing_table.columns)])
#forcing_table = forcing_table.round(decimals = 2)

In [9]:
s = forcing_table.style.format(precision=2)
s.set_table_styles([
        {'selector': 'toprule', 'props': ':hline;'},
        {'selector': 'midrule', 'props': ':hline;'},
        {'selector': 'bottomrule', 'props': ':hline;'}])
print(s.to_latex(column_format= 'lccccc'))

\begin{tabular}{lccccc}
\hline
 & F2x & F0p5x & F4x & F4x/F2x & F4x/F0p5x \\
\hline
CESM2 & 4.56 & -5.82 & 9.22 & 2.02 & -1.58 \\
CNRM-CM6-1 & 4.23 & -3.74 & 8.29 & 1.96 & -2.22 \\
CanESM5 & 3.69 & -3.55 & 7.57 & 2.05 & -2.13 \\
GISS-E2-1-G & 3.92 & -3.78 & 8.01 & 2.04 & -2.12 \\
GISS-E2-1-H & 3.98 & nan & 8.16 & 2.05 & nan \\
GISS-E2-2-G & 3.67 & nan & 7.51 & 2.05 & nan \\
GISS-E2-2-H & 3.36 & nan & 7.06 & 2.10 & nan \\
IPSL-CM6A-LR & 4.23 & -3.17 & 9.28 & 2.19 & -2.93 \\
MIROC6 & 3.27 & -3.20 & 7.61 & 2.32 & -2.38 \\
MRI-ESM2-0 & 3.57 & -4.45 & 7.71 & 2.16 & -1.73 \\
TaiESM1 & 4.48 & -4.51 & 9.76 & 2.18 & -2.17 \\
HadGEM3-GC31-LL & 3.73 & -3.04 & 7.92 & 2.12 & -2.60 \\
Model mean & 3.89 & -3.92 & 8.17 & 2.10 & -2.21 \\
\hline
\end{tabular}



## Overview of 4xCO2 estimates

In [2]:
best_parameters_4xCO2 = pd.read_csv('../Estimates/best_estimated_parameters2_allmembers4xCO2.csv', index_col=0).sort_index()
F4x = pd.DataFrame(2*best_parameters_4xCO2['F2x']).rename(columns={"F2x": "F4x"})
T4x = pd.DataFrame(2*best_parameters_4xCO2['T2x']).rename(columns={"T2x": "T4x"})
#F4x.index.name = 'Models'

# pick columns to show in table
tau = best_parameters_4xCO2[['tau1','tau2','tau3']].rename(columns={"tau1": "$\tau_1$", "tau2": "$\tau_2$", "tau3": "$\tau_3$"})
a_n = best_parameters_4xCO2[['a_1','a_2','a_3']]
b_n = best_parameters_4xCO2[['b_1','b_2','b_3']]
lambda_n = pd.DataFrame({'$-\lambda_1$': b_n['b_1']/a_n['a_1'], '$-\lambda_2$': b_n['b_2']/a_n['a_2'], '$-\lambda_3$': b_n['b_3']/a_n['a_3']})
lambda_n;
b_4 = best_parameters_4xCO2[['b_4']]

all_cols = tau.join([lambda_n, b_4, F4x, T4x])

# add model mean row in the bottom:
model_means = [all_cols.iloc[:,i].mean() for i in range(0,len(all_cols.columns))]
all_cols = pd.concat([all_cols,pd.DataFrame([model_means], index = ['Model mean'], columns = all_cols.columns)])


In [3]:
print(len(all_cols.index))
all_cols.index

54


Index(['ACCESS-CM2', 'ACCESS-ESM1-5', 'AWI-CM-1-1-MR', 'BCC-CSM2-MR',
       'BCC-ESM1', 'CAMS-CSM1-0', 'CAS-ESM2-0', 'CESM2', 'CESM2-FV2',
       'CESM2-WACCM', 'CESM2-WACCM-FV2', 'CIESM', 'CMCC-CM2-SR5', 'CMCC-ESM2',
       'CNRM-CM6-1', 'CNRM-CM6-1-HR', 'CNRM-ESM2-1', 'CanESM5', 'E3SM-1-0',
       'EC-Earth3', 'EC-Earth3-AerChem', 'EC-Earth3-CC', 'EC-Earth3-Veg',
       'FGOALS-f3-L', 'FGOALS-g3', 'GFDL-CM4', 'GFDL-ESM4', 'GISS-E2-1-G',
       'GISS-E2-1-H', 'GISS-E2-2-G', 'GISS-E2-2-H', 'HadGEM3-GC31-LL',
       'HadGEM3-GC31-MM', 'ICON-ESM-LR', 'IITM-ESM', 'INM-CM4-8', 'INM-CM5-0',
       'IPSL-CM5A2-INCA', 'IPSL-CM6A-LR', 'KIOST-ESM', 'MIROC-ES2L', 'MIROC6',
       'MPI-ESM-1-2-HAM', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0',
       'NESM3', 'NorCPM1', 'NorESM2-LM', 'NorESM2-MM', 'SAM0-UNICON',
       'TaiESM1', 'UKESM1-0-LL', 'Model mean'],
      dtype='object')

In [4]:
all_cols;

In [5]:
s2 = all_cols.style.format(precision=2)
s2.set_table_styles([
        {'selector': 'toprule', 'props': ':hline;'},
        {'selector': 'midrule', 'props': ':hline;'},
        {'selector': 'bottomrule', 'props': ':hline;'}])
#print(s2.to_latex(column_format= 'lccccccccc'))
print(s2.to_latex())

\begin{tabular}{lrrrrrrrrr}
\hline
 & $	au_1$ & $	au_2$ & $	au_3$ & $-\lambda_1$ & $-\lambda_2$ & $-\lambda_3$ & b_4 & F4x & T4x \\
\hline
ACCESS-CM2 & 2.57 & 25.28 & 139.03 & 1.58 & 0.60 & 0.62 & 1.63 & 8.93 & 9.98 \\
ACCESS-ESM1-5 & 2.63 & 14.26 & 324.14 & 1.41 & 1.02 & 0.46 & 0.37 & 7.42 & 9.40 \\
AWI-CM-1-1-MR & 1.17 & 5.99 & 87.70 & 1.72 & 1.38 & 1.07 & 0.58 & 8.62 & 6.42 \\
BCC-CSM2-MR & 1.11 & 7.18 & 184.44 & 1.93 & 1.25 & 0.88 & 0.11 & 7.74 & 6.30 \\
BCC-ESM1 & 2.72 & 19.25 & 337.09 & 1.32 & 0.94 & 0.75 & 0.02 & 6.96 & 6.96 \\
CAMS-CSM1-0 & 1.87 & 10.32 & 131.09 & 2.07 & 1.91 & 1.61 & 0.09 & 8.92 & 4.67 \\
CAS-ESM2-0 & 1.43 & 8.93 & 81.05 & 2.06 & 1.07 & 0.85 & 0.87 & 8.81 & 7.11 \\
CESM2 & 1.38 & 6.93 & 291.55 & 1.80 & 1.11 & 0.44 & 0.06 & 9.22 & 12.15 \\
CESM2-FV2 & 1.15 & 6.14 & 381.16 & 4.43 & 0.93 & 0.35 & 0.12 & 10.78 & 13.02 \\
CESM2-WACCM & 1.13 & 5.75 & 287.90 & 2.11 & 1.17 & 0.49 & 0.20 & 9.01 & 10.97 \\
CESM2-WACCM-FV2 & 1.03 & 5.54 & 427.24 & -0.00 & 1.00 & 0.40 & 0

## Data overview

In [43]:
forcingdirectory = '../Estimates/Transient_forcing_estimates/'
model_names = [ f.name for f in os.scandir(forcingdirectory) if f.is_dir() and f.name !='.ipynb_checkpoints']
model_names.sort()
model_names;

In [44]:
trans_experiment_list = ['1pctCO2', 'historical', 'hist-nat', 'hist-GHG', 'hist-aer', 'ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
abrupt_exp = ['abrupt-4xCO2', 'abrupt-2xCO2', 'abrupt-0p5xCO2'] 
piClim_exp = ['piClim-4xCO2', 'piClim-histall']
all_experiment_list = trans_experiment_list + abrupt_exp + piClim_exp
all_experiment_list

['1pctCO2',
 'historical',
 'hist-nat',
 'hist-GHG',
 'hist-aer',
 'ssp119',
 'ssp126',
 'ssp245',
 'ssp370',
 'ssp585',
 'abrupt-4xCO2',
 'abrupt-2xCO2',
 'abrupt-0p5xCO2',
 'piClim-4xCO2',
 'piClim-histall']

In [45]:
df_all = pd.DataFrame('-', index = model_names, columns = all_experiment_list); # will contain number of transient forcing estimates

# transient forcing dataframe:
dff = pd.DataFrame(); # will contain number of transient forcing estimates

for model in model_names:
    for exp in trans_experiment_list:
        forcingsubdir = os.path.join(forcingdirectory, model, exp)
        if os.path.isdir(forcingsubdir):
            forcingfiles = [ f.name for f in os.scandir(forcingsubdir) if f.name !='.ipynb_checkpoints']
            dff.loc[model, exp] = int(len(forcingfiles)/3)
        else:
            dff.loc[model, exp] = '-'
            
df_all[trans_experiment_list] = dff

headers = list(np.tile('Experiments with transient forcing', len(dff.columns)))\
        + list(np.tile('Abrupt CO2 experiments', len(abrupt_exp))) \
        + list(np.tile('Fixed-SST experiments', len(piClim_exp)))
df_all.columns = pd.MultiIndex.from_tuples(list(zip(headers, df_all.columns)))
df_all;

In [46]:
#### fill in number of piClim-4xCO2 experiments: ####

fSST_df = pd.read_csv('../Estimates/fixed_SST_forcing_estimates.csv', index_col = 0)
fSSTmodels = fSST_df.index.unique()
for model in fSSTmodels:
    nmembers = len(fSST_df.index[fSST_df.index == model])
    df_all['Fixed-SST experiments', 'piClim-4xCO2'].loc[model] = nmembers

    
#### fill in number of piClim-histall experiments: ####
transientERF_files = [ f.name for f in os.scandir('../Estimates/piClim-histall_forcing/') if f.name !='.ipynb_checkpoints']

model_names_transERF = [file.rsplit('_')[0] for file in transientERF_files]
unique_model_names = set(model_names_transERF)
for model in unique_model_names:
    df_all['Fixed-SST experiments', 'piClim-histall'].loc[model] = model_names_transERF.count(model)

/var/folders/vh/7gh1pj993l18k240624j8frr0000gn/T/ipykernel_12731/361775822.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all['Fixed-SST experiments', 'piClim-4xCO2'].loc[model] = nmembers
/var/folders/vh/7gh1pj993l18k240624j8frr0000gn/T/ipykernel_12731/361775822.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all['Fixed-SST experiments', 'piClim-histall'].loc[model] = model_names_transERF.count(model)


In [47]:
#### fill in number of abrupt CO2 experiments: ####
directory = os.path.join('../Processed_data/Global_annual_anomalies')

for model in model_names:
    modeldirectory = os.path.join(directory, model)
    modelexp_names = [ f.name for f in os.scandir(modeldirectory) if f.is_dir() and f.name !='.ipynb_checkpoints']
    for exp in abrupt_exp:
        if exp in modelexp_names:
            # almost all anomalies are used:
            members = find_members(model, exp, datatype = 'anomalies')
            if model == 'GISS-E2-1-G' and exp == 'abrupt-4xCO2':
                members.remove('r1i1p1f3')
                print(members)
            #print(model, exp, members)
            df_all['Abrupt CO2 experiments', exp].loc[model] = int(len(members))  
        #else:
        #    df_all['Abrupt CO2 experiments', exp].loc[model] = '-'


/var/folders/vh/7gh1pj993l18k240624j8frr0000gn/T/ipykernel_12731/4103716112.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all['Abrupt CO2 experiments', exp].loc[model] = int(len(members))


['r102i1p1f1', 'r1i1p1f1', 'r1i1p3f1', 'r1i1p5f1']


In [48]:
df_all
# but this turned out to be way too big to fit in the paper

Experiments with transient forcing                      \
                                             1pctCO2 historical hist-nat   
ACCESS-CM2                                       1.0        5.0      3.0   
ACCESS-ESM1-5                                    1.0       40.0      3.0   
AWI-CM-1-1-MR                                    1.0        5.0        -   
BCC-CSM2-MR                                      1.0        3.0        3   
BCC-ESM1                                         1.0        3.0        -   
CAMS-CSM1-0                                      2.0        3.0        -   
CESM2                                            1.0       11.0        3   
CESM2-FV2                                        1.0        3.0        -   
CESM2-WACCM                                      1.0        3.0        -   
CESM2-WACCM-FV2                                  1.0        3.0        -   
CMCC-CM2-SR5                                     1.0        7.0        -   
CMCC-ESM2                                        1.0        1.0        -   
CNRM-CM6-1                                       1.0       29.0       10   
CNRM-CM6-1-HR                                    1.0        1.0        -   
CNRM-ESM2-1                                     10.0       11.0        -   
CanESM5                                          6.0       65.0       50   
E3SM-1-0                                         1.0        5.0        -   
EC-Earth3                                        1.0       72.0        -   
EC-Earth3-AerChem                                1.0        2.0        -   
EC-Earth3-CC                                     1.0        1.0        -   
EC-Earth3-Veg                                    1.0        9.0        -   
FGOALS-f3-L                                      3.0        3.0        -   
FGOALS-g3                                        3.0        6.0        3   
GFDL-CM4                                         1.0        1.0        3   
GFDL-ESM4                                        1.0        1.0        3   
GISS-E2-1-G                                      5.0       46.0       20   
GISS-E2-1-H                                      1.0       25.0        -   
GISS-E2-2-G                                      1.0          -        -   
GISS-E2-2-H                                      1.0          5        -   
HadGEM3-GC31-LL                                  4.0          5       10   
HadGEM3-GC31-MM                                  1.0          4        -   
ICON-ESM-LR                                      1.0          5        -   
IITM-ESM                                         1.0          1        -   
INM-CM4-8                                        1.0          1        -   
INM-CM5-0                                        1.0         10        -   
IPSL-CM5A2-INCA                                  1.0          1        -   
IPSL-CM6A-LR                                     1.0         33       10   
KIOST-ESM                                        1.0          1        -   
MIROC-ES2L                                       1.0         26        -   
MIROC6                                           1.0         50       50   
MPI-ESM-1-2-HAM                                  1.0          3        -   
MPI-ESM1-2-HR                                    1.0         10        -   
MPI-ESM1-2-LR                                    1.0         30        -   
MRI-ESM2-0                                       2.0         12        5   
NESM3                                            1.0          5        -   
NorCPM1                                          1.0         30        -   
NorESM2-LM                                       1.0          3        3   
NorESM2-MM                                       1.0          3        -   
SAM0-UNICON                                      1.0          1        -   
TaiESM1                                          1.0          2        -   
UKESM1-0-LL                                      4.0         19   

In [74]:
# count #models for each experiment
exp_count = pd.DataFrame('-', index = ['Number of models'], columns=df_all.columns)
for exp in df_all.columns:
    count = len(df_all[exp])-list(df_all[exp]).count('-')
    exp_count[exp] = count
    
df_all = pd.concat([df_all, exp_count])

In [75]:
exp_count

Experiments with transient forcing                      \
                                            1pctCO2 historical hist-nat   
Number of models                                 51         50       15   

                                                                       \
                 hist-GHG hist-aer ssp119 ssp126 ssp245 ssp370 ssp585   
Number of models       14       14     15     38     38     37     39   

                 Abrupt CO2 experiments                              \
                           abrupt-4xCO2 abrupt-2xCO2 abrupt-0p5xCO2   
Number of models                     51           12              9   

                 Fixed-SST experiments                 
                          piClim-4xCO2 piClim-histall  
Number of models                    18             10

In [76]:
df_all

Experiments with transient forcing                      \
                                             1pctCO2 historical hist-nat   
ACCESS-CM2                                       1.0        5.0      3.0   
ACCESS-ESM1-5                                    1.0       40.0      3.0   
AWI-CM-1-1-MR                                    1.0        5.0        -   
BCC-CSM2-MR                                      1.0        3.0        3   
BCC-ESM1                                         1.0        3.0        -   
CAMS-CSM1-0                                      2.0        3.0        -   
CESM2                                            1.0       11.0        3   
CESM2-FV2                                        1.0        3.0        -   
CESM2-WACCM                                      1.0        3.0        -   
CESM2-WACCM-FV2                                  1.0        3.0        -   
CMCC-CM2-SR5                                     1.0        7.0        -   
CMCC-ESM2                                        1.0        1.0        -   
CNRM-CM6-1                                       1.0       29.0       10   
CNRM-CM6-1-HR                                    1.0        1.0        -   
CNRM-ESM2-1                                     10.0       11.0        -   
CanESM5                                          6.0       65.0       50   
E3SM-1-0                                         1.0        5.0        -   
EC-Earth3                                        1.0       72.0        -   
EC-Earth3-AerChem                                1.0        2.0        -   
EC-Earth3-CC                                     1.0        1.0        -   
EC-Earth3-Veg                                    1.0        9.0        -   
FGOALS-f3-L                                      3.0        3.0        -   
FGOALS-g3                                        3.0        6.0        3   
GFDL-CM4                                         1.0        1.0        3   
GFDL-ESM4                                        1.0        1.0        3   
GISS-E2-1-G                                      5.0       46.0       20   
GISS-E2-1-H                                      1.0       25.0        -   
GISS-E2-2-G                                      1.0          -        -   
GISS-E2-2-H                                      1.0          5        -   
HadGEM3-GC31-LL                                  4.0          5       10   
HadGEM3-GC31-MM                                  1.0          4        -   
ICON-ESM-LR                                      1.0          5        -   
IITM-ESM                                         1.0          1        -   
INM-CM4-8                                        1.0          1        -   
INM-CM5-0                                        1.0         10        -   
IPSL-CM5A2-INCA                                  1.0          1        -   
IPSL-CM6A-LR                                     1.0         33       10   
KIOST-ESM                                        1.0          1        -   
MIROC-ES2L                                       1.0         26        -   
MIROC6                                           1.0         50       50   
MPI-ESM-1-2-HAM                                  1.0          3        -   
MPI-ESM1-2-HR                                    1.0         10        -   
MPI-ESM1-2-LR                                    1.0         30        -   
MRI-ESM2-0                                       2.0         12        5   
NESM3                                            1.0          5        -   
NorCPM1                                          1.0         30        -   
NorESM2-LM                                       1.0          3        3   
NorESM2-MM                                       1.0          3        -   
SAM0-UNICON                                      1.0          1        -   
TaiESM1                                          1.0          2        -   
UKESM1-0-LL                                      4.0         19   

In [77]:

s3 = df_all[['Abrupt CO2 experiments']].style.format(precision=0)
s3.set_table_styles([
        {'selector': 'toprule', 'props': ':hline;'},
        {'selector': 'midrule', 'props': ':hline;'},
        {'selector': 'bottomrule', 'props': ':hline;'}])
print(s3.to_latex(column_format= 'lccc'))
#print(s3.to_latex())

\begin{tabular}{lccc}
\hline
 & \multicolumn{3}{r}{Abrupt CO2 experiments} \\
 & abrupt-4xCO2 & abrupt-2xCO2 & abrupt-0p5xCO2 \\
\hline
ACCESS-CM2 & 1 & - & - \\
ACCESS-ESM1-5 & 2 & - & - \\
AWI-CM-1-1-MR & 1 & - & - \\
BCC-CSM2-MR & 1 & - & - \\
BCC-ESM1 & 1 & - & - \\
CAMS-CSM1-0 & 2 & - & - \\
CESM2 & 1 & 1 & 1 \\
CESM2-FV2 & 1 & - & - \\
CESM2-WACCM & 1 & - & - \\
CESM2-WACCM-FV2 & 1 & - & - \\
CMCC-CM2-SR5 & 1 & - & - \\
CMCC-ESM2 & 1 & - & - \\
CNRM-CM6-1 & 6 & 1 & 1 \\
CNRM-CM6-1-HR & 1 & - & - \\
CNRM-ESM2-1 & 3 & - & - \\
CanESM5 & 2 & 1 & 1 \\
E3SM-1-0 & 1 & - & - \\
EC-Earth3 & 2 & - & - \\
EC-Earth3-AerChem & 1 & - & - \\
EC-Earth3-CC & 1 & - & - \\
EC-Earth3-Veg & 1 & - & - \\
FGOALS-f3-L & 3 & - & - \\
FGOALS-g3 & 1 & - & - \\
GFDL-CM4 & 1 & - & - \\
GFDL-ESM4 & 1 & - & - \\
GISS-E2-1-G & 4 & 4 & 1 \\
GISS-E2-1-H & 3 & 2 & - \\
GISS-E2-2-G & 1 & 1 & - \\
GISS-E2-2-H & 1 & 1 & - \\
HadGEM3-GC31-LL & 1 & 1 & 1 \\
HadGEM3-GC31-MM & 1 & - & - \\
ICON-ESM-LR & 1 & - & - \\
IIT

In [80]:
#s3 = df_all['Experiments with transient forcing'].style.format(precision=0)
s3 = df_all['Fixed-SST experiments'].loc[list(fSSTmodels)].style.format(precision=0)
s3.set_table_styles([
        {'selector': 'toprule', 'props': ':hline;'},
        {'selector': 'midrule', 'props': ':hline;'},
        {'selector': 'bottomrule', 'props': ':hline;'}])
print(s3.to_latex(column_format= 'lcc'))
#print(s3.to_latex(column_format= 'lccccccccccccccc'))

\begin{tabular}{lcc}
\hline
 & piClim-4xCO2 & piClim-histall \\
\hline
ACCESS-CM2 & 1 & - \\
ACCESS-ESM1-5 & 1 & - \\
CESM2 & 1 & - \\
CNRM-CM6-1 & 1 & 1 \\
CNRM-ESM2-1 & 1 & - \\
CanESM5 & 1 & 3 \\
EC-Earth3 & 1 & 1 \\
GFDL-CM4 & 1 & 3 \\
GFDL-ESM4 & 1 & - \\
GISS-E2-1-G & 2 & 3 \\
HadGEM3-GC31-LL & 1 & 3 \\
IPSL-CM6A-LR & 5 & 3 \\
MIROC6 & 1 & 3 \\
MPI-ESM1-2-LR & 1 & 1 \\
MRI-ESM2-0 & 1 & - \\
NorESM2-LM & 2 & 6 \\
NorESM2-MM & 1 & - \\
UKESM1-0-LL & 1 & - \\
\hline
\end{tabular}

